In [5]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
#  home = '/Users/mahtin'

somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "testpon")

static = os.path.join(home, "Dropbox/Icke/Work/static")
shell_path = "../shell"
# squeezing all data and shell paths into config
# shell paths can be autogenerated from common shell path
mawk_tools = ["cleanpileup", "csv2bed", "pon2cols", "pile2count", "filterVar", "pon2tumor"]
mawk_tool_dict = {tool:f"{shell_path}/{tool}.mawk" for tool in mawk_tools}


EBconfig = {
    "makeponlist": "../shell/makeponlist.sh",
    "pon_path": pon_path,
    "genome_split": f"{static}/genome/gatk/hg38/split",
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"}
}
EBconfig.update(mawk_tool_dict)
EBconfig

{'makeponlist': '../shell/makeponlist.sh',
 'pon_path': '/Users/martinscience/Dropbox/Icke/Work/somVar/testdata/testpon',
 'genome_split': '/Users/martinscience/Dropbox/Icke/Work/static/genome/gatk/hg38/split',
 'MAPQ': 20,
 'Q': 25,
 'fit_pen': 0.5,
 'count_dict': {0: 'alt+', 1: 'alt-', 2: 'depth+', 3: 'depth-'},
 'cleanpileup': '../shell/cleanpileup.mawk',
 'csv2bed': '../shell/csv2bed.mawk',
 'pon2cols': '../shell/pon2cols.mawk',
 'pile2count': '../shell/pile2count.mawk',
 'filterVar': '../shell/filterVar.mawk',
 'pon2tumor': '../shell/pon2tumor.mawk'}

In [6]:
from ebraw import bam2matrix
bam_file = f"{testdata}/bam/002_A.bam"
mut_file = f"{ebdata}/002.csv"
pon_list = "Pon_chr7.txt"
chrom = "chr7"
df = bam2matrix(bam_file, mut_file, chrom, pon_list, EBconfig)
df[:3]

../code/ebutils.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  pon_df['sample'] = pon_df['basename'].str.replace(


08:10:55 : 1 matches target bam /Users/martinscience/Dropbox/Icke/Work/somVar/testdata/bam/002_A.bam and is removed from pon_list. If not desired, change names in PoN
                                                 bam
0  /Users/martinscience/Dropbox/Icke/Work/somVar/...
1  /Users/martinscience/Dropbox/Icke/Work/somVar/...
3  /Users/martinscience/Dropbox/Icke/Work/somVar/...
4  /Users/martinscience/Dropbox/Icke/Work/somVar/...
5  /Users/martinscience/Dropbox/Icke/Work/somVar/...
$ samtools mpileup -Q 25 -q 20 -l /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/EBdata/002_chr7.bed -f /Users/martinscience/Dropbox/Icke/Work/static/genome/gatk/hg38/split/chr7.fa -b /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/EBdata/002_chr7.pon -r chr7 | cut -f $(../shell/pon2cols.mawk /Users/martinscience/Dropbox/Icke/Work/somVar/testdata/testpon/Pon_chr7.txt) | ../shell/cleanpileup.mawk | ../shell/pile2count.mawk | ../shell/filterVar.mawk /Users/martinscience/Dropbox/Icke/Work/somVar/to

,Chr,Start,End,Ref,Alt,Tumor:Alt=Depth,PON:ALT=Depth
0,chr7,680651,680651,T,G,0-1=28-2,0|0|0|0-5|2|3|3=129|92|148|166-10|6|16|8
1,chr7,35670232,35670232,C,T,4-3=13-4,102|0|0|0-18|0|0|0=102|92|167|229-18|24|42|44
2,chr7,72826970,72826970,G,A,2-5=10-10,40|0|0|99-26|0|0|50=126|189|187|291-80|100|110...


### EX|INport

In [ ]:
# get the code
import sys
sys.path.append('../code')

# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
ebdata = f"{testdata}/EBdata"
pon_path = f"{testdata}/testpon"
static = f"{home}/Dropbox/Icke/Work/static"


# df.to_csv(f"{ebdata}/002.matrix", sep='\t', index=False)
df = pd.read_csv(f"{ebdata}/002.matrix", sep='\t')
df[:3]

## matrix to AB

### get_count_df for matrix2EB

In [ ]:
from ebutils import get_count_df
row = df.iloc[2,:]
row
cdf = get_count_df(row['PON:ALT=Depth'], EBconfig["count_dict"])
cdf

In [ ]:
from ebconvert import matrix2AB
from ebcore import fit_bb
bb_params = fit_bb(cdf, 0.5)
bb_params

In [ ]:
matrix2AB(row, config=EBconfig)

### expand matrix2AB to df

In [ ]:
df['ABparams'] = df.apply(matrix2AB, config=EBconfig, axis=1)
df[:3]

## AB to EB

In [ ]:
# make a row with the same values as in EBold for comparison
row_dict = {
    "Chr": ["chr7"],
    "Start": 680651,
    "Tumor:Alt=Depth": "0-5=12-42",
    "ABparams": "0.20509|23.07746-0.22661|2.47235"
}

test_row = pd.DataFrame().from_dict(row_dict).iloc[0]
test_row
from ebconvert import AB2EBscore
AB2EBscore(test_row, config=EBconfig)

In [ ]:
df['EBscore'] = df.apply(AB2EBscore, config=EBconfig, axis=1)
df